In [1]:
import os
import numpy as np
import pandas as pd
import sklearn
from sklearn import multioutput
from sklearn import multiclass
from sklearn import metrics
from sklearn import svm

In [2]:
os.chdir('C:/Users/tku/Desktop/data/finish')
filename = 'm.csv'
df = pd.read_csv(filename)
df_dum = pd.get_dummies(df)
list(df_dum)

['chinese',
 'english',
 'math',
 'society',
 'science',
 'PR',
 'H_GPA',
 'M',
 'M_10_A',
 'M_10_B',
 'M_10_C',
 'M_10_D',
 'M_10_E',
 'M_10_F',
 'M_10_G',
 'M_10_H',
 'M_10_I',
 'M_10_J',
 'M_5_A',
 'M_5_B',
 'M_5_C',
 'M_5_D',
 'M_5_F',
 'M_3_A',
 'M_3_B',
 'M_3_F',
 'M_2_A',
 'M_2_F']

In [3]:
train_set, test_set = \
              np.split(df_dum.sample(frac=1, random_state=1), 
                       [int(.9*len(df_dum))])

In [4]:
X_train = train_set[['chinese', 'english', 'math', 'society', 'science', 'PR', 'H_GPA']]
#X_validate = validate_set[['chinese', 'english', 'math', 'society', 'science', 'PR', 'H_GPA']]
X_test = test_set[['chinese', 'english', 'math', 'society', 'science', 'PR', 'H_GPA']]

Y_train10= train_set[['M_10_A', 'M_10_B', 'M_10_C', 'M_10_D', 'M_10_E', 'M_10_F', 'M_10_G', 'M_10_H', 'M_10_I', 'M_10_J']]
#Y_validate10= validate_set[['M_10_A', 'M_10_B', 'M_10_C', 'M_10_D', 'M_10_E', 'M_10_F', 'M_10_G', 'M_10_H', 'M_10_I', 'M_10_J']]
Y_test10= test_set[['M_10_A', 'M_10_B', 'M_10_C', 'M_10_D', 'M_10_E', 'M_10_F', 'M_10_G', 'M_10_H', 'M_10_I', 'M_10_J']]

Y_train5= train_set[['M_5_A', 'M_5_B', 'M_5_C', 'M_5_D', 'M_5_F']]
#Y_validate5= validate_set[['M_5_A', 'M_5_B', 'M_5_C', 'M_5_D', 'M_5_F']]
Y_test5= test_set[['M_5_A', 'M_5_B', 'M_5_C', 'M_5_D', 'M_5_F']]

Y_train3= train_set[['M_3_A', 'M_3_B', 'M_3_F']]
#Y_validate3= validate_set[['M_3_A', 'M_3_B', 'M_3_F']]
Y_test3= test_set[['M_3_A', 'M_3_B', 'M_3_F']]

Y_train2= train_set[['M_2_A', 'M_2_F']]
#Y_validate2= validate_set[['M_2_A', 'M_2_F']]
Y_test2= test_set[['M_2_A', 'M_2_F']]


# SVM

In [5]:
kernel = 'poly'
shape = 'ovr'
degree = 3
C=1

In [6]:
if kernel == 'poly':
    clf10 = svm.SVC(kernel=kernel,C=C,decision_function_shape=shape,degree=degree)
    clf5 = svm.SVC(kernel=kernel,C=C,decision_function_shape=shape,degree=degree)
    clf3 = svm.SVC(kernel=kernel,C=C,decision_function_shape=shape,degree=degree)
    clf2 = svm.SVC(kernel=kernel,C=C,decision_function_shape=shape,degree=degree)
else:
    clf10 = svm.SVC(kernel=kernel,C=C,decision_function_shape=shape)
    clf5 = svm.SVC(kernel=kernel,C=C,decision_function_shape=shape)
    clf3 = svm.SVC(kernel=kernel,C=C,decision_function_shape=shape)
    clf2 = svm.SVC(kernel=kernel,C=C,decision_function_shape=shape)
if shape == 'ovo':
    print("in ovo")
    clfm10 = multiclass.OneVsOneClassifier(clf10)
    clfm5 = multiclass.OneVsOneClassifier(clf5)
    clfm3 = multiclass.OneVsOneClassifier(clf3)
    clfm2 = multiclass.OneVsOneClassifier(clf2)
else:
    print("in ovr")
    clfm10 = multiclass.OneVsRestClassifier(clf10)
    clfm5 = multiclass.OneVsRestClassifier(clf5)
    clfm3 = multiclass.OneVsRestClassifier(clf3)
    clfm2 = multiclass.OneVsRestClassifier(clf2)
tclfm10 = multioutput.MultiOutputClassifier(clfm10)
tclfm5 = multioutput.MultiOutputClassifier(clfm5)
tclfm3 = multioutput.MultiOutputClassifier(clfm3)
tclfm2= multioutput.MultiOutputClassifier(clfm2)

in ovr


In [7]:
tclfm10.fit(X_train, Y_train10)
tclfm5.fit(X_train, Y_train5)
tclfm3.fit(X_train, Y_train3)
tclfm2.fit(X_train, Y_train2)

MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=SVC(C=1,
                                                                  kernel='poly')))

In [8]:
pred10= tclfm10.predict(X_test)
pred5= tclfm5.predict(X_test)
pred3= tclfm3.predict(X_test)
pred2= tclfm2.predict(X_test)
score10 = metrics.accuracy_score(Y_test10,pred10)*100
score5 = metrics.accuracy_score(Y_test5,pred5)*100
score3 = metrics.accuracy_score(Y_test3,pred3)*100
score2 = metrics.accuracy_score(Y_test2,pred2)*100

In [9]:
Y_pred10c = pred10
Y_test10c = Y_test10.to_numpy()
Y_pred5c = pred5
Y_test5c = Y_test5.to_numpy()

In [10]:
oneAway10 =1
            
for i in range(Y_pred10c.shape[0]):
    for j in range(Y_pred10c.shape[1]):
        if Y_test10c[i,j] == 1:
            if j == 0:
                if Y_pred10c[i,j] == 1 or Y_pred10c[i,j+1] == 1:
                    oneAway10 = oneAway10 +1
            elif j == Y_pred10c.shape[1]-1:
                if Y_pred10c[i,j-1] == 1 or Y_pred10c[i,j] == 1:
                    oneAway10 = oneAway10 +1
            else:
                if Y_pred10c[i,j-1] == 1 or Y_pred10c[i,j] or Y_pred10c[i,j+1] == 1:
                    oneAway10 = oneAway10 +1
            break
print("oneAway accurency of 10class =%.2f%%" % (oneAway10/Y_pred10c.shape[0]*100))

oneAway5 =1
            
for i in range(Y_pred5c.shape[0]):
    for j in range(Y_pred5c.shape[1]):
        if Y_test5c[i,j] == 1:
            if j == 0:
                if Y_pred5c[i,j] == 1 or Y_pred5c[i,j+1] == 1:
                    oneAway5 = oneAway5 +1
            elif j == 3:
                if Y_pred5c[i,j-1] == 1 or Y_pred5c[i,j] == 1:
                    oneAway5 = oneAway5 +1
            elif j == 4:
                if Y_pred5c[i,j] == 1:
                    oneAway5 = oneAway5 +1
            else:
                if Y_pred5c[i,j-1] == 1 or Y_pred5c[i,j] or Y_pred5c[i,j+1] == 1:
                    oneAway5 = oneAway5 +1
            break
print("oneAway accurency of 5class =%.2f%%" % (oneAway5/Y_pred5c.shape[0]*100))

oneAway accurency of 10class =3.23%
oneAway accurency of 5class =22.58%


In [11]:
from tempfile import NamedTemporaryFile
import shutil
import csv


fields = ['m','c10_accur','c10_oneAway','c5_accur','c5_oneAway','c3_accur','c2_accur']
writelist = ["SVM","m",
             "%.2f%%" % (metrics.accuracy_score(Y_test10,pred10)*100),"%.2f%%" % (oneAway10/Y_pred10c.shape[0]*100),
             "%.2f%%" % (metrics.accuracy_score(Y_test5,pred5)*100),"%.2f%%" % (oneAway5/Y_pred5c.shape[0]*100),
             "%.2f%%" % (metrics.accuracy_score(Y_test3,pred3)*100),"%.2f%%" % (metrics.accuracy_score(Y_test2,pred2)*100)]

with open('result.csv', 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(writelist)
    csvfile.close()